In [1]:
%matplotlib widget

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm

In [3]:
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

**AWS null 처리**

In [48]:
folder = 'data'
file_name = 'AWS_옥산_2016.xlsx'
path = os.path.join(folder, file_name)
# not needed anymore
if False:
    df = pd.read_excel(path)
    df_new = df[df['stn_name'].notna()]
    df_new.to_excel(path, index=False)

**구미보 interpollation**

In [154]:
# not needed anymore
if False:
    file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
    folder = 'data'
    for file_name in file_names:
        path = os.path.join(folder, file_name)
        df = pd.read_excel(path)
        df.loc[df['upstrim_wlv']==0, 'upstrim_wlv'] = np.nan
        df.loc[df['dwstrm_wlv']==0, 'dwstrm_wlv'] = np.nan
        df['upstrim_wlv'].interpolate(limit_direction='both', inplace=True)
        df['dwstrm_wlv'].interpolate(limit_direction='both', inplace=True)
        stn_name = df.loc[df['stn_name'].notna(), 'stn_name'].iloc[0]
        stn_name
        df.loc[df['stn_name'].isna(), 'stn_name'] = stn_name
        df.to_excel(path, index=False)

**GAIN**

**낙동1**

In [4]:
file_names = [
              ['자동_해평_2016.xlsx',
               '자동_해평_2017.xlsx',
               '자동_해평_2018.xlsx',
               '자동_해평_2019.xlsx'],
              ['자동_도개_2016.xlsx',
               '자동_도개_2017.xlsx',
               '자동_도개_2018.xlsx',
               '자동_도개_2019.xlsx'],
              ['자동_신암_2016.xlsx',
               '자동_신암_2017.xlsx',
               '자동_신암_2018.xlsx',
               '자동_신암_2019.xlsx'],
             ]
gan_files = [ '자동_해평_gan.xlsx', '자동_도개_gan.xlsx', '자동_신암_gan.xlsx']
ori_files = [ '자동_해평_org.xlsx', '자동_도개_org.xlsx', '자동_신암_org.xlsx']
folder = 'data'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    if loc == 0:
        df.append(df_full[loc].iloc[:, 2:9].copy())
    else:
        df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]].copy())
    date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)


자동_해평_2016.xlsx
자동_해평_2017.xlsx
자동_해평_2018.xlsx
자동_해평_2019.xlsx
자동_도개_2016.xlsx
자동_도개_2017.xlsx
자동_도개_2018.xlsx
자동_도개_2019.xlsx
자동_신암_2016.xlsx
자동_신암_2017.xlsx
자동_신암_2018.xlsx
자동_신암_2019.xlsx


**hour df 추출**

In [5]:
file_names =  ['도개_2016.xlsx',
               '도개_2017.xlsx',
               '도개_2018.xlsx',
               '도개_2019.xlsx']

#file_name = file_names[0]

folder = 'water_data/낙동강1/자동'

date_dfs = []
for i in range(len(file_names)):
    path = os.path.join(folder, file_names[i])
    date_dfs.append(pd.read_excel(path, index_col='ymdh'))
    date_dfs[i] = pd.DataFrame(date_dfs[i], columns=[])

date_df = pd.concat(date_dfs)

In [ ]:
folder = 'data'
path = os.path.join(folder, 'date.xlsx')
date_df.to_excel(path, index=False)

In [89]:
file_names = [
              ['상주1_2016.xlsx',
               '상주1_2017.xlsx',
               '상주1_2018.xlsx',
               '상주1_2019.xlsx'],
              ['영강2-1_2016.xlsx',
               '영강2-1_2017.xlsx',
               '영강2-1_2018.xlsx',
               '영강2-1_2019.xlsx'],
             ]
folder = 'water_data/낙동강2/수질측정망'

filename = '상주1_2016.xlsx'

# what's nh3n, no3n, ss, po4p, dtn, dtp?
qm_cols = ['nh3n', 'no3n', 'ph', 'cod', 'ec', 'bod', 'ss', 'do', 'tn', 'tp', 'chlorophylla', 'toc']
#df = df.loc[:, qm_cols]

qm_dfs = []
qm_inter_dfs = []
for j in range(len(file_names)):
    dfs = []
    for i in range(len(file_names[j])):
        path = os.path.join(folder, file_names[j][i])
        dfs.append(pd.read_excel(path, index_col='dt'))
        dfs[i] = dfs[i].loc[:, qm_cols]
    qm_dfs.append(pd.concat(dfs))
    qm_dfs[j] = pd.concat([date_df, qm_dfs[j]], axis=1)
    qm_dfs[j].reset_index(drop=True, inplace=True)
    first_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    last_series = qm_dfs[j].apply(pd.Series.first_valid_index)
    for col in first_series.index:
        qm_dfs[j].loc[0, col] = qm_dfs[j].loc[first_series[col], col]
        qm_dfs[j].loc[len(qm_dfs[j])-1, col] = qm_dfs[j].loc[last_series[col], col]
    
    qm_dfs[j] = qm_dfs[j].interpolate(method='pchip')

In [90]:
a = qm_dfs[0].loc[:, 'chlorophylla'].to_numpy()
plt.figure()
#plt.scatter(qm_dfs[0].loc[:, 'chlorophylla'].index, qm_dfs[0].loc[:, 'chlorophylla'])
plt.plot(qm_dfs[0].loc[:, 'chlorophylla'])
#plt.plot(qm_inter_dfs[0].loc[:, 'chlorophylla'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Normalize and rename columns**

In [91]:
qm_mean = []
qm_std = []

for i in range(len(qm_dfs)):
    qm_mean.append(qm_dfs[i].mean())
    qm_std.append(qm_dfs[i].std())
    qm_dfs[i] = (qm_dfs[i]-qm_mean[i])/qm_std[i]
    qm_dfs[i].columns=qm_dfs[i].columns+'%d'%(i+1)

In [93]:
qm_dfs[1]

nh3n2     no3n2       ph2      cod2       ec2      bod2       ss2  \
0     -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   
1     -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   
2     -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   
3     -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   
4     -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -0.301985  0.946456 -0.342790 -0.511940  0.663851 -0.485053 -0.495966   
35060 -0.300651  0.941306 -0.341693 -0.505642  0.666814 -0.482285 -0.495966   
35061 -0.299315  0.936137 -0.340597 -0.499326  0.669783 -0.479500 -0.495966   
35062 -0.297980  0.930947 -0.339502 -0.492992  0.672756 -0.476700 -0.495966   
35063 -0.296644  0.925738 -0.338408 -0.486640  0.675733 -0.473884 -0.495966   

            do2       tn2       tp2  chlorophylla2      toc2  
0      1.842839  0.466005 -0.466784      -0.830421 -0.958855  
1      1.842839  0.466005 -0.466784      -0.830421 -0.958855  
2      1.842839  0.466005 -0.466784      -0.830421 -0.958855  
3      1.842839  0.466005 -0.466784      -0.830421 -0.958855  
4      1.842839  0.466005 -0.466784      -0.830421 -0.958855  
...         ...       ...       ...            ...       ...  
35059  1.844883  0.471203 -0.462744      -0.832820 -0.951830  
35060  1.844378  0.469918 -0.463753      -0.832227 -0.953584  
35061  1.843869  0.468623 -0.464762      -0.831629 -0.955339  
35062  1.843356  0.467319 -0.465773      -0.831027 -0.957096  
35063  1.842839  0.466005 -0.466784      -0.830421 -0.958855  

[35064 rows x 12 columns]

**낙동강2 자동측정망**

In [158]:
file_names = [
              ['낙동강1/자동/도개_2016.xlsx',
               '낙동강1/자동/도개_2017.xlsx',
               '낙동강1/자동/도개_2018.xlsx',
               '낙동강1/자동/도개_2019.xlsx'],
              ['낙동강1/자동/신암_2016.xlsx',
               '낙동강1/자동/신암_2017.xlsx',
               '낙동강1/자동/신암_2018.xlsx',
               '낙동강1/자동/신암_2019.xlsx'],
              ['낙동강1/자동/회상_2016.xlsx',
               '낙동강1/자동/회상_2017.xlsx',
               '낙동강1/자동/회상_2018.xlsx',
               '낙동강1/자동/회상_2019.xlsx'],
#              ['낙동강2/자동측정망/풍양_2016.xlsx',
#               '낙동강2/자동측정망/풍양_2017.xlsx',
#               '낙동강2/자동측정망/풍양_2018.xlsx',
#               '낙동강2/자동측정망/풍양_2019.xlsx'],
             ]
#gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx', '풍양_gan.xlsx']
#ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx', '풍양_gan.xlsx']
gan_files = [ '도개_gan.xlsx', '신암_gan.xlsx', '회상_gan.xlsx']
ori_files = [ '도개_org.xlsx', '신암_org.xlsx', '회상_org.xlsx']
folder = 'water_data'

day = 24*60*60
year = (365.2425)*day

df_full = []
df = []

auto_cols = ['tmpr_value', 'ph_value', 'do_value', 'ec_value', 'toc_value', '총질소_값', '총인_값', '클로로필-a_값']

for loc in range(len(file_names)):
    
    df_loc = []
    for y in range(len(file_names[loc])):
        path = os.path.join(folder, file_names[loc][y])
        print(file_names[loc][y])
        df_loc.append(pd.read_excel(path))
    df_full.append(pd.concat(df_loc))
    df.append(df_full[loc].loc[:, auto_cols].copy())
    date_time = pd.to_datetime(df_full[loc].loc[:, 'ymdh'], format='%Y.%m.%d %H:%M', utc=True)
    timestamp_s = date_time.map(datetime.datetime.timestamp)
    df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
    df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
    df[loc] = df[loc].reset_index(drop=True)

낙동강1/자동/도개_2016.xlsx
낙동강1/자동/도개_2017.xlsx
낙동강1/자동/도개_2018.xlsx
낙동강1/자동/도개_2019.xlsx
낙동강1/자동/신암_2016.xlsx
낙동강1/자동/신암_2017.xlsx
낙동강1/자동/신암_2018.xlsx
낙동강1/자동/신암_2019.xlsx
낙동강1/자동/회상_2016.xlsx
낙동강1/자동/회상_2017.xlsx
낙동강1/자동/회상_2018.xlsx
낙동강1/자동/회상_2019.xlsx
낙동강2/자동측정망/풍양_2016.xlsx
낙동강2/자동측정망/풍양_2017.xlsx
낙동강2/자동측정망/풍양_2018.xlsx
낙동강2/자동측정망/풍양_2019.xlsx


In [159]:
df[0].iloc[21310:21360, :]

tmpr_value  ph_value  do_value   ec_value  toc_value    총질소_값     총인_값  \
21310    25.77500   9.25000  11.80000  222.93892    4.57733      NaN      NaN   
21311    26.00000   9.23333  11.35000  223.40000    4.43833      NaN      NaN   
21312    26.07500   9.20000  11.02500  224.23333    4.37317      NaN      NaN   
21313    25.79167   9.24167  11.44167  223.11392    4.15167      NaN      NaN   
21314    25.85833   9.23333  11.25000  223.11942    4.11833      NaN      NaN   
21315    25.90000   9.22500  11.04167  223.31392    3.95200      NaN      NaN   
21316    25.78000   9.19167  11.14167  223.22217    3.93933      NaN      NaN   
21317    25.48889   9.20000  11.17500  222.78883    3.91800      NaN      NaN   
21318    25.82000   9.20000  10.60000  223.91108    3.79900      NaN      NaN   
21319    25.92222   9.20000  10.69167  224.00000    3.70833      NaN      NaN   
21320    25.95833   9.20000  10.74167  223.99158    3.81867      NaN      NaN   
21321    26.27500   9.20000  10.58333  223.99725    3.68200      NaN      NaN   
21322    26.63750   9.20000  10.62500  224.00000    3.56550      NaN      NaN   
21323         NaN       NaN       NaN        NaN    3.67220  2.45300  0.01400   
21324    27.24167   9.28333  11.77500  223.37500    3.51817  2.42450  0.01217   
21325    27.20000   9.30000  12.10833  223.71392    3.50350  2.48708  0.00900   
21326    27.20000   9.30000  12.02500  223.99725    3.42517  2.38608  0.00983   
21327    27.24545   9.30000  12.16667  223.11975    3.50833  2.42767  0.00758   
21328    27.31111   9.30000  12.07500  223.01133    3.46583  2.35900  0.00708   
21329    27.37500   9.30000  12.10833  222.97225    3.36300  2.32358  0.00883   
21330    27.20833   9.30000  12.34167  222.98608    3.22550  2.37367  0.00767   
21331    27.10000   9.30000  12.05833  223.00000    3.25383  2.44925  0.01008   
21332    27.44167   9.30000  12.14167  223.14167    3.19817  2.39850  0.01042   
21333    27.22500   9.30000  12.28333  222.99725    3.12683  2.27017  0.01000   
21334    27.09167   9.30000  12.27500  222.91392    3.08617  2.26275  0.00958   
21335    27.15833   9.30000  11.88333  224.06383    3.04217  2.27300  0.00933   
21336    26.91667   9.28333  11.85000  223.80008    3.02250  2.29308  0.01025   
21337    26.69167   9.30000  11.88333  222.19167    3.06950  2.24950  0.01142   
21338    26.54167   9.30000  12.00833  222.19442    3.02033  2.27450  0.01058   
21339    26.55833   9.30000  11.71667  222.97492    3.03733  2.32083  0.00975   
21340    26.35833   9.30000  11.67500  222.01942    2.96400  2.29942  0.00883   
21341    26.19167   9.30000  12.03333  222.97225    2.95317  2.29700  0.00692   
21342    26.04167   9.29167  11.65833  222.86383    2.95417  2.24375  0.00783   
21343    26.03333   9.30000  11.47500  223.01100    2.98600  2.25725  0.00817   
21344    26.16364   9.30000  11.78333  223.89925    3.01533  2.17633  0.00825   
21345    26.32727   9.30000  11.40833  223.69608    3.00500  2.21225  0.00950   
21346    26.56667   9.30000  11.53333  223.35283    3.01767  2.19400  0.00950   
21347    26.80833   9.30000  11.59167  223.77783    3.02333  2.17650  0.00950   
21348    26.92500   9.30000  11.42500  222.99725    2.99067  2.21100  0.01050   
21349    27.13333   9.33333  11.93333  222.43058    2.97517  2.21733  0.01150   
21350    27.26667   9.30000  11.65000  223.35283    2.99667  2.15775  0.01100   
21351         NaN       NaN       NaN        NaN    3.07571  2.15000      NaN   
21352    27.06667   9.24167  11.75833  224.33892    3.04033  2.19820  0.01300   
21353    26.93333   9.20000  11.49167  224.56108    3.08850  2.19908  0.01150   
21354    26.90000   9.20000  11.65833  224.93617    3.02217  2.21417  0.01500   
21355    26.65000   9.20000  12.35000  224.61667    3.10583  2.24850  0.01600   
21356    26.56667   9.20000  12.29167  224.00000    3.12183  2.22350  0.01450   
21357    26.59167   9.25000  12.21667  224.15008    3.09333  2.20892  0.01650   
21358    26.43333   9.20000  11.85000 

In [160]:
# normalize data

df_all = pd.concat(df)
df_all

train_mean = df_all.mean()
train_std = df_all.std()
for i in range(len(file_names)):
    df[i] = (df[i]-train_mean)/train_std

In [161]:
df[0]*train_std + train_mean

tmpr_value  ph_value  do_value   ec_value  toc_value    총질소_값     총인_값  \
0         4.70000   7.80000  11.10000  333.01442    1.35833      NaN  0.00357   
1         4.70000   7.79167  11.09167  333.00000    1.32500      NaN  0.00683   
2         4.70000   7.80000  11.03333  333.00000    1.34167      NaN  0.00700   
3         4.70000   7.80000  11.03333  333.00000    1.34167      NaN  0.00542   
4         4.67500   7.80000  10.99167  333.00000    1.38333      NaN  0.00475   
...           ...       ...       ...        ...        ...      ...      ...   
35059     3.80000   8.10000  10.06250  266.77500    2.18917  3.13025  0.00625   
35060     3.80000   8.10000  10.06167  265.07500    2.27500  3.19292  0.00467   
35061     3.75000   8.10000  10.04667  263.15000    2.30750  3.18625  0.00558   
35062     3.61667   8.10000  10.03833  264.21667    2.18167  3.17975  0.00858   
35063     3.57500   8.10000  10.03917  263.98333    2.19000  3.14450  0.00542   

       클로로필-a_값       Day sin   Day cos  Year sin  Year cos  
0       6.65000 -1.407890e-12  1.000000 -0.002666  0.999996  
1       6.83333  2.588190e-01  0.965926 -0.001950  0.999998  
2       7.01667  5.000000e-01  0.866025 -0.001233  0.999999  
3       6.98333  7.071068e-01  0.707107 -0.000516  1.000000  
4       7.13333  8.660254e-01  0.500000  0.000201  1.000000  
...         ...           ...       ...       ...       ...  
35059  58.10833 -9.659258e-01  0.258819 -0.005734  0.999984  
35060  58.80833 -8.660254e-01  0.500000 -0.005017  0.999987  
35061  57.43333 -7.071068e-01  0.707107 -0.004301  0.999991  
35062  57.70833 -5.000000e-01  0.866025 -0.003584  0.999994  
35063  57.65833 -2.588190e-01  0.965926 -0.002867  0.999996  

[35064 rows x 12 columns]

In [162]:
#총 location 수
n_loc = len(df)
# 그림 col 수
n_col = n_loc*2
plt.figure(figsize=(9,10))
isnan = np.isnan(df_all).astype(int)
#isnan = np.isnan(df[0]).astype(int)
data = isnan.to_numpy()
n = data.shape[0]
seq_len = n//n_col
for i in range(n_col):
    plt.subplot(101+(n_col*10)+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:8], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [15]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [16]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [17]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [18]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [19]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [20]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [21]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [22]:
def normalization (data, parameters=None):
  '''Normalize data in [0, 1] range.
  
  Args:
    - data: original data
  
  Returns:
    - norm_data: normalized data
    - norm_parameters: min_val, max_val for each feature for renormalization
  '''

  # Parameters
  _, dim = data.shape
  norm_data = data.copy()

  if parameters is None:

    # MixMax normalization
    min_val = np.zeros(dim)
    max_val = np.zeros(dim)
   
    # For each dimension
    for i in range(dim):
      min_val[i] = np.nanmin(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
      max_val[i] = np.nanmax(norm_data[:,i])
      norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

    # Return norm_parameters for renormalization
    norm_parameters = {'min_val': min_val,
                       'max_val': max_val}
  else:
    min_val = parameters['min_val']
    max_val = parameters['max_val']

    # For each dimension
    for i in range(dim):
      norm_data[:,i] = norm_data[:,i] - min_val[i]
      norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

    norm_parameters = parameters

  return norm_data, norm_parameters

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
        #print(group)
        n = np.max(group)
        #print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        #print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)

miss_data file saved


In [25]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [26]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)
            
        
        # normlize for spam
        if normalize:
            self.data, norm_param = normalization(self.data)
        #print(norm_param)
        
        # Define mask matrix
        if miss_pattern is None:
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            self.miss = MissData(load_dir='save')
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

In [27]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp + 0.5*(1-H_temp)
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save', prefix=''):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, prefix+'discriminator.h5')
        gen_savefile = os.path.join(save_dir, prefix+'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save', prefix=''):
        disc_savefile = os.path.join(save_dir, prefix+'discriminator.h5')
        gen_savefile = os.path.join(save_dir, prefix+'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [28]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

In [29]:
train_df = df_all
val_df = df_all
test_df = df_all

In [30]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)
_ = wide_window.train
wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

In [31]:
#wide_window.plot(plot_col='클로로필-a')
wide_window.plot(plot_col='클로로필-a_값')
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  17409


In [32]:
val_performance = {}
performance = {}

In [33]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [34]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [35]:
#model.fit를 사용하지 않을 때에는 학습 데이터 로딩
#gain.load(save_dir='save')
gain.load(save_dir='save', prefix='nagdong2')

model weights loaded


In [496]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)


#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                    patience=2,
#                                                    mode='min')
#gain.compile()

Epoch 1/2000
1/1 [==============================] - 1s 582ms/step - gen_loss: 108.8035 - disc_loss: 0.7251 - rmse: 1.0077 - val_loss: 0.9039
Epoch 2/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 95.1376 - disc_loss: 0.5474 - rmse: 0.9609 - val_loss: 0.7869
Epoch 3/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 76.5279 - disc_loss: 0.4921 - rmse: 0.8631 - val_loss: 0.7863
Epoch 4/2000
1/1 [==============================] - 0s 131ms/step - gen_loss: 64.8732 - disc_loss: 0.3987 - rmse: 0.7578 - val_loss: 0.7135
Epoch 5/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 53.1697 - disc_loss: 0.3982 - rmse: 0.7097 - val_loss: 0.6956
Epoch 6/2000
1/1 [==============================] - 0s 157ms/step - gen_loss: 51.5318 - disc_loss: 0.3906 - rmse: 0.7107 - val_loss: 0.6431
Epoch 7/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 40.3224 - disc_loss: 0.3953 - rmse: 0.6661 - val_loss: 0.6525
Epoch 8/2000
1/1 [=

1/1 [==============================] - 0s 145ms/step - gen_loss: 3.5547 - disc_loss: 0.2448 - rmse: 0.2440 - val_loss: 0.2579
Epoch 118/2000
1/1 [==============================] - 0s 96ms/step - gen_loss: 3.7675 - disc_loss: 0.2532 - rmse: 0.2476 - val_loss: 0.2760
Epoch 119/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 3.7102 - disc_loss: 0.2510 - rmse: 0.2432 - val_loss: 0.2596
Epoch 120/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 3.7632 - disc_loss: 0.2464 - rmse: 0.2514 - val_loss: 0.2756
Epoch 121/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 3.6581 - disc_loss: 0.2484 - rmse: 0.2345 - val_loss: 0.2984
Epoch 122/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 3.6535 - disc_loss: 0.2404 - rmse: 0.2679 - val_loss: 0.2461
Epoch 123/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 3.5670 - disc_loss: 0.2447 - rmse: 0.2499 - val_loss: 0.2746
Epoch 124/2000
1/1 [=========

1/1 [==============================] - 0s 116ms/step - gen_loss: 2.8793 - disc_loss: 0.1968 - rmse: 0.2503 - val_loss: 0.2517
Epoch 234/2000
1/1 [==============================] - 0s 115ms/step - gen_loss: 3.2936 - disc_loss: 0.1973 - rmse: 0.2655 - val_loss: 0.2496
Epoch 235/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.8902 - disc_loss: 0.1950 - rmse: 0.2465 - val_loss: 0.2460
Epoch 236/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 2.6458 - disc_loss: 0.1988 - rmse: 0.2197 - val_loss: 0.2420
Epoch 237/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 2.9349 - disc_loss: 0.2004 - rmse: 0.2501 - val_loss: 0.2709
Epoch 238/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 2.7654 - disc_loss: 0.1927 - rmse: 0.2264 - val_loss: 0.2563
Epoch 239/2000
1/1 [==============================] - 0s 132ms/step - gen_loss: 2.7474 - disc_loss: 0.1984 - rmse: 0.2426 - val_loss: 0.2275
Epoch 240/2000
1/1 [========

1/1 [==============================] - 0s 113ms/step - gen_loss: 2.7124 - disc_loss: 0.1778 - rmse: 0.2151 - val_loss: 0.2336
Epoch 350/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 2.5485 - disc_loss: 0.1780 - rmse: 0.2281 - val_loss: 0.2170
Epoch 351/2000
1/1 [==============================] - 0s 116ms/step - gen_loss: 2.5238 - disc_loss: 0.1778 - rmse: 0.2241 - val_loss: 0.2325
Epoch 352/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 2.5989 - disc_loss: 0.1747 - rmse: 0.2195 - val_loss: 0.2387
Epoch 353/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 2.6497 - disc_loss: 0.1828 - rmse: 0.2126 - val_loss: 0.2281
Epoch 354/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 2.6333 - disc_loss: 0.1770 - rmse: 0.2229 - val_loss: 0.2395
Epoch 355/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 2.4311 - disc_loss: 0.1821 - rmse: 0.2219 - val_loss: 0.2481
Epoch 356/2000
1/1 [========

1/1 [==============================] - 0s 133ms/step - gen_loss: 2.0680 - disc_loss: 0.1703 - rmse: 0.2110 - val_loss: 0.2271
Epoch 466/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 2.2833 - disc_loss: 0.1637 - rmse: 0.2125 - val_loss: 0.2133
Epoch 467/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 2.3861 - disc_loss: 0.1667 - rmse: 0.2171 - val_loss: 0.2279
Epoch 468/2000
1/1 [==============================] - 0s 98ms/step - gen_loss: 2.2773 - disc_loss: 0.1668 - rmse: 0.2358 - val_loss: 0.2400
Epoch 469/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.3010 - disc_loss: 0.1680 - rmse: 0.2321 - val_loss: 0.2327
Epoch 470/2000
1/1 [==============================] - 0s 162ms/step - gen_loss: 2.1585 - disc_loss: 0.1689 - rmse: 0.2303 - val_loss: 0.2365
Epoch 471/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.2838 - disc_loss: 0.1667 - rmse: 0.2250 - val_loss: 0.2151
Epoch 472/2000
1/1 [=========

1/1 [==============================] - 0s 141ms/step - gen_loss: 2.2662 - disc_loss: 0.1583 - rmse: 0.2500 - val_loss: 0.2269
Epoch 582/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 2.1005 - disc_loss: 0.1529 - rmse: 0.2198 - val_loss: 0.2332
Epoch 583/2000
1/1 [==============================] - 0s 171ms/step - gen_loss: 2.5326 - disc_loss: 0.1505 - rmse: 0.2912 - val_loss: 0.2243
Epoch 584/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.2197 - disc_loss: 0.1524 - rmse: 0.2456 - val_loss: 0.2326
Epoch 585/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 2.3233 - disc_loss: 0.1549 - rmse: 0.2366 - val_loss: 0.2451
Epoch 586/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 2.2718 - disc_loss: 0.1529 - rmse: 0.2565 - val_loss: 0.2224
Epoch 587/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 2.2185 - disc_loss: 0.1548 - rmse: 0.2217 - val_loss: 0.2416
Epoch 588/2000
1/1 [========

1/1 [==============================] - 0s 138ms/step - gen_loss: 2.3882 - disc_loss: 0.1390 - rmse: 0.2356 - val_loss: 0.2388
Epoch 698/2000
1/1 [==============================] - 0s 107ms/step - gen_loss: 2.3893 - disc_loss: 0.1447 - rmse: 0.2403 - val_loss: 0.2376
Epoch 699/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 2.3127 - disc_loss: 0.1397 - rmse: 0.2294 - val_loss: 0.2265
Epoch 700/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 2.2158 - disc_loss: 0.1439 - rmse: 0.2146 - val_loss: 0.2134
Epoch 701/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.3049 - disc_loss: 0.1416 - rmse: 0.2217 - val_loss: 0.2390
Epoch 702/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 2.3108 - disc_loss: 0.1397 - rmse: 0.2202 - val_loss: 0.2172
Epoch 703/2000
1/1 [==============================] - 0s 112ms/step - gen_loss: 2.1971 - disc_loss: 0.1414 - rmse: 0.2147 - val_loss: 0.2151
Epoch 704/2000
1/1 [========

1/1 [==============================] - 0s 126ms/step - gen_loss: 1.9454 - disc_loss: 0.1364 - rmse: 0.1964 - val_loss: 0.2002
Epoch 814/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 1.8802 - disc_loss: 0.1311 - rmse: 0.2170 - val_loss: 0.2151
Epoch 815/2000
1/1 [==============================] - 0s 155ms/step - gen_loss: 2.0735 - disc_loss: 0.1380 - rmse: 0.2167 - val_loss: 0.1956
Epoch 816/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 1.9520 - disc_loss: 0.1402 - rmse: 0.2021 - val_loss: 0.2184
Epoch 817/2000
1/1 [==============================] - 0s 404ms/step - gen_loss: 1.9127 - disc_loss: 0.1410 - rmse: 0.1959 - val_loss: 0.2296
Epoch 818/2000
1/1 [==============================] - 0s 108ms/step - gen_loss: 2.2445 - disc_loss: 0.1432 - rmse: 0.2138 - val_loss: 0.2096
Epoch 819/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 2.1561 - disc_loss: 0.1376 - rmse: 0.2242 - val_loss: 0.2413
Epoch 820/2000
1/1 [========

In [497]:
#gain.save(save_dir='save_paldang')
#gain.save(prefix='nagdong')
gain.save(prefix='nagdong2')

In [36]:
#wide_window.plot(gain, plot_col='클로로필-a')
wide_window.plot(gain, plot_col='클로로필-a_값')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## GAIN 추론

In [37]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape =', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n =', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x_block = x[0:n].copy()
    most_shape = x_block.shape
    x_block = x_block.reshape((-1,)+unit_shape)
    x_block_remain = x[-time_seq:].copy()
    remain_shape = x_block_remain.shape
    x_block_remain = x_block_remain.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_remain = gain.predict(x_block_remain)
    y_gan = y.reshape(most_shape)
    y_remain_gan = y_remain.reshape(remain_shape)
    y_gan = np.append(y_gan, y_remain_gan[-(total_n-n):], axis=0)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
    print(y_gan.shape)

105192
unit_shape = (120, 12)
120
n = 105120
(35064, 12)
(35064, 8)
(35064, 8)


In [38]:
gan_dfs = []
ori_dfs = []
for i in range(len(gans)):
    gan_dfs.append(pd.DataFrame(gans[i], columns=df[i].columns[:(gans[i].shape[1])]))
    ori_dfs.append(pd.DataFrame(oris[i], columns=df[i].columns[:(gans[i].shape[1])]))

**Save data**

In [39]:
folder = 'data'
prefix = 'nagdong2'
def save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix):
    for i in range(len(gan_dfs)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs[i].to_excel(path, index=False)
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs[i].to_excel(path, index=False)
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std.to_frame(name='std').to_excel(path)
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean.to_frame(name='mean').to_excel(path)

def load_gan_data(gan_files, ori_files, prefix):
    gan_dfs = []
    ori_dfs = []
    for i in range(len(gan_files)):
        path = os.path.join(folder, gan_files[i])
        gan_dfs.append(pd.read_excel(path))
        print(path)
        path = os.path.join(folder, ori_files[i])
        ori_dfs.append(pd.read_excel(path))
    path = os.path.join(folder, prefix + '_std.xlsx')
    train_std = pd.read_excel(path, index_col=0).loc[:, 'std']
    path = os.path.join(folder, prefix + '_mean.xlsx')
    train_mean = pd.read_excel(path, index_col=0).loc[:, 'mean']
    return gan_dfs, ori_dfs, train_mean, train_std

In [40]:
save_gan_data(gan_dfs, ori_dfs, gan_files, ori_files, train_mean, train_std, prefix=prefix)

data/도개_gan.xlsx
data/신암_gan.xlsx
data/회상_gan.xlsx


**Load GAN data**

In [507]:
gan_dfs2, ori_dfs2, train_mean2, train_std2 = load_gan_data(gan_files, ori_files, prefix=prefix)

data/도개_gan.xlsx
data/신암_gan.xlsx
data/회상_gan.xlsx


In [41]:
gan_nan_dfs = []
for i in range(len(gan_dfs)):
    gan_nan_dfs.append(gan_dfs[i].copy())
    isnan = ori_dfs[i].isna()
    gan_nan_dfs[i][~isnan] = np.nan

In [42]:
n = len(auto_cols)
for j in range(len(gan_dfs)):
    plt.figure(figsize=(9,20))
    for i in range(n):
        #plt.subplot('%d1%d'%(n,i))
        plt.subplot((n*100)+11+i)
        plt.plot(ori_dfs[j].iloc[:, i])
        plt.plot(gan_nan_dfs[j].iloc[:, i])
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
file_names = []
file_names = ['보_구미보_2016.xlsx', '보_구미보_2017.xlsx', '보_구미보_2018.xlsx', '보_구미보_2019.xlsx']
folder = 'data'
bo_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    bo_dfs.append(pd.read_excel(path).iloc[:, 2:4])
bo_df = pd.concat(bo_dfs, axis=0)

bo_mean = bo_df.mean()
bo_std = bo_df.std()
bo_df = (bo_df - bo_mean)/bo_std

bo_df.reset_index(drop=True, inplace=True)

data/보_구미보_2016.xlsx
data/보_구미보_2017.xlsx
data/보_구미보_2018.xlsx
data/보_구미보_2019.xlsx


In [44]:
file_names = []
file_names = ['AWS_옥산_2016.xlsx', 'AWS_옥산_2017.xlsx', 'AWS_옥산_2018.xlsx', 'AWS_옥산_2019.xlsx']
folder = 'data'
aws_dfs = []
for file_name in file_names:
    path = os.path.join(folder, file_name)
    print(path)
    aws_dfs.append(pd.read_excel(path).iloc[:, -4:])
aws_df = pd.concat(aws_dfs, axis=0)

aws_mean = aws_df.mean()
aws_std = aws_df.std()
aws_df = (aws_df - aws_mean)/aws_std
aws_df.reset_index(drop=True, inplace=True)

data/AWS_옥산_2016.xlsx
data/AWS_옥산_2017.xlsx
data/AWS_옥산_2018.xlsx
data/AWS_옥산_2019.xlsx


In [95]:
mean = pd.concat([train_mean, bo_mean, aws_mean]+qm_mean)
std = pd.concat([train_std, bo_std, aws_std]+qm_std)
mean

tmpr_value      1.619773e+01
ph_value        7.957707e+00
do_value        1.028830e+01
ec_value        2.361925e+02
toc_value       2.949259e+00
총질소_값           2.511900e+00
총인_값            1.923296e-02
클로로필-a_값        1.821510e+01
Day sin         6.037103e-15
Day cos         2.747682e-15
Year sin       -5.681266e-08
Year cos        2.053380e-05
upstrim_wlv     3.246092e+03
dwstrm_wlv      2.571552e+03
rn60m_value     7.638889e-02
ta_value        8.758325e+00
hm_value        2.880299e+01
ps_value        7.607794e+02
nh3n            7.441017e-02
no3n            1.995208e+00
ph              7.986643e+00
cod             5.426084e+00
ec              2.315260e+02
bod             1.595262e+00
ss              6.677707e+00
do              1.144842e+01
tn              2.423511e+00
tp              2.772457e-02
chlorophylla    1.183009e+01
toc             2.633367e+00
nh3n            2.100124e-01
no3n            1.963437e+00
ph              7.703737e+00
cod             3.973896e+00
ec            

# 상관관계 분석

In [96]:
from tqdm import tqdm

**Feature combine**

In [97]:
ori_df = pd.concat(ori_dfs, axis=1)
gan_df = pd.concat(gan_dfs, axis=1)

ori_df = pd.concat([ori_df, bo_df, aws_df]+qm_dfs, axis=1)
gan_df = pd.concat([gan_df, bo_df, aws_df]+qm_dfs, axis=1)

total_no = ori_df.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan_df[train_slice])
val_df = pd.DataFrame(gan_df[val_slice])
test_df = pd.DataFrame(gan_df[test_slice])

train_ori_df = pd.DataFrame(ori_df[train_slice])
val_ori_df = pd.DataFrame(ori_df[val_slice])
test_ori_df = pd.DataFrame(ori_df[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
out_features = [6]
out_num_features = len(out_features)
out_num_features

1

**지점간 동일 측정항목 비교**

In [98]:
total_lag = 24*5
#total_lag = 5
#index1 = 3 # 가평 chl-a
#label1 = '가평 chl-a'
#index2 = 3+11 # 의암호 chl-a
#label2 = '의암호 chl-a'
n_col = len(auto_cols)

data = gan_df

# source 0,1,2
source = 1

n = data.shape[0]

corr = np.empty((n_col, total_lag,))
#for col in range(1):
for col in range(n_col):
#for col in range(2):
    for lag in tqdm(range(total_lag)):
        index1 = col
        index2 = col+4+(source+1)*n_col
        x1 = data.iloc[:n-lag, index1].to_numpy()
        x2 = data.iloc[lag:, index2].to_numpy()
        #df = pd.DataFrame([x1,x2], index=[index1, index2])
        #df = df.transpose()
        #df_corr = df.corr(method='pearson')
        #corr[col,lag] = df_corr.iloc[0,1]
        correl = np.corrcoef(x1, x2)
        corr[col,lag] = correl[0,1]
        

100%|██████████| 120/120 [00:00<00:00, 1691.46it/s]


In [99]:
cols = ori_df.columns
plt.figure(figsize=(9, 12))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()
#plt.plot(corr[0], label=cols[0])
#plt.plot(corr[1], label=cols[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [989]:
cols = ori_df.columns
plt.figure(figsize=(9, 12))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()
#plt.plot(corr[0], label=cols[0])
#plt.plot(corr[1], label=cols[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Source 측정요소와 Target 측정요소 간의 correlation 분석**

In [100]:
total_lag = 24*20
#total_lag = 5
#index1 = 3 # 가평 chl-a
#label1 = '가평 chl-a'
#index2 = 3+11 # 의암호 chl-a
#label2 = '의암호 chl-a'

data = gan_df

# target index
target_index = 7
# source index
n_col = data.shape[1]

n = data.shape[0]

corr = np.empty((n_col, total_lag,))
#for col in range(1):
for col in range(n_col):
#for col in range(1):
    for lag in tqdm(range(total_lag)):
        index1 = target_index
        index2 = col
        x1 = data.iloc[:n-lag, index1].to_numpy()
        x2 = data.iloc[lag:, index2].to_numpy()
        #df = pd.DataFrame([x1,x2], index=[index1, index2])
        #df = df.transpose()
        #df_corr = df.corr(method='pearson')
        #corr[col,lag] = df_corr.iloc[0,1]
        correl = np.corrcoef(x1, x2)
        corr[col,lag] = correl[0, 1]

100%|██████████| 480/480 [00:00<00:00, 2218.21it/s]


In [101]:
cols = ori_df.columns
plt.figure(figsize=(9, 50))
for i in range(n_col):
    plt.subplot(n_col, 1, i+1)
    plt.plot(corr[i], label=cols[i])
    plt.legend(prop=fprop)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
plt.figure()
plt.plot(ori_df.iloc[:, 6])
plt.plot(ori_df.iloc[:, 17])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Random forest와 Boruta를 이용한 feature 분석

**source-target data creation (target-가평, source-의암호) or (해평, 도개)**

In [103]:
hours = 24
test_avg = test_df.groupby(test_df.index //24).mean()
total_no= test_avg.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)
train_avg = test_avg[train_slice]
val_avg = test_avg[val_slice]
#val_avg_df = pd.DataFrame(gan[val_slice], columns=columns)
#test_avg_df = pd.DataFrame(gan[test_slice], columns=columns)


In [104]:
val_avg

tmpr_value  ph_value  do_value  ec_value  toc_value     총질소_값      총인_값  \
1022    0.000949 -0.201438  0.098882 -0.202792   0.224014  1.006254  0.619287   
1023   -0.016363 -0.121580 -0.047344 -0.116359  -0.104946  0.790503  0.429691   
1024   -0.033624  0.056111 -0.012608  0.002342  -0.339983  0.775980  0.293586   
1025    0.099229 -0.120004  0.174691 -0.028313  -0.568288  0.722338  0.194843   
1026    0.045743 -0.055692  0.240293  0.106820  -0.646255  0.778109  0.156265   
...          ...       ...       ...       ...        ...       ...       ...   
1456   -1.238548  0.143428 -0.139302  0.813617  -0.829944  0.660292 -0.487560   
1457   -1.251615  0.279505 -0.060242  0.638319  -0.868311  0.643182 -0.558583   
1458   -1.283107  0.267927 -0.114121  0.941113  -0.819372  0.657083 -0.647777   
1459   -1.281060  0.317135 -0.111458  0.904859  -0.758125  0.704852 -0.623632   
1460   -1.326125  0.240718 -0.084714  0.700491  -0.801417  0.830300 -0.593229   

      클로로필-a_값       Day sin       Day cos  ...       ph2      cod2       ec2  \
1022 -0.548428  3.013886e-13 -3.024248e-13  ... -0.081477 -1.125367 -0.535164   
1023 -0.635017 -5.560737e-13 -3.024248e-13  ...  0.307311 -1.097193 -0.479162   
1024 -0.562865 -5.560737e-13  5.550375e-13  ...  0.752168 -1.056862 -0.433050   
1025 -0.521549  3.013886e-13 -3.024248e-13  ...  1.171540 -1.010198 -0.393423   
1026 -0.324710  3.013145e-13 -3.023878e-13  ...  1.483871 -0.963023 -0.356873   
...        ...           ...           ...  ...       ...       ...       ...   
1456  1.636630  3.013886e-13 -3.024248e-13  ... -0.460148 -1.058014  0.390270   
1457  1.699134 -5.560737e-13  5.550375e-13  ... -0.432278 -0.950797  0.446698   
1458  2.279866  3.013886e-13  5.550745e-13  ... -0.404798 -0.831406  0.507882   
1459  2.622409  3.013886e-13 -3.023878e-13  ... -0.377720 -0.700353  0.573163   
1460  2.414202 -5.561477e-13 -3.024248e-13  ... -0.351054 -0.558149  0.641882   

          bod2       ss2       do2       tn2       tp2  chlorophylla2  \
1022 -1.055465 -0.390106  0.188969 -0.003890 -0.601550      -0.756020   
1023 -1.002118 -0.398216  0.240881 -0.011343 -0.619148      -0.756020   
1024 -0.948764 -0.404745  0.287467 -0.021130 -0.635907      -0.756020   
1025 -0.895714 -0.409570  0.330785 -0.030964 -0.650073      -0.756020   
1026 -0.843278 -0.412569  0.372889 -0.038560 -0.659895      -0.756020   
...        ...       ...       ...       ...       ...            ...   
1456 -0.693612 -0.495966  1.879581  0.559424 -0.366395      -0.873548   
1457 -0.657816 -0.495966  1.874263  0.545901 -0.387014      -0.867305   
1458 -0.614820 -0.495966  1.867431  0.528531 -0.408811      -0.859286   
1459 -0.564054 -0.495966  1.858897  0.506832 -0.431612      -0.849269   
1460 -0.504947 -0.495966  1.848470  0.480323 -0.455242      -0.837031   

          toc2  
1022 -1.123511  
1023 -1.097507  
1024 -1.061633  
1025 -1.022402  
1026 -0.986331  
...        ...  
1456 -0.784309  
1457 -0.820257  
1458 -0.858154  
1459 -0.897750  
1460 -0.938793  

[439 rows x 58 columns]

## Data Generator

In [105]:
from tqdm import tqdm

In [106]:
class RFDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size = None,
                 input_width = 24*3,
                 label_width = 1,
                 shift = 24*1,
                 skip_time = None,
                 shuffle = False,
                 out_features = None,
                 out_num_features = 1,
                 selected_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if batch_size:
            self.batch_size = batch_size
        else:
            self.batch_size = self.no
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        self.selected_features = selected_features
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        #x = np.empty((0, self.input_width, self.data.shape[1]))
        self.derived_column_features = 6
        if self.selected_features is None: # all features
            x = np.empty((self.batch_size, (self.input_width+self.derived_column_features)*self.data.shape[1]))
        else:
            x = np.empty((self.batch_size, np.sum(boruta_features.astype(int))))
        y = np.empty((self.batch_size,))
        for cnt in tqdm(range(0, self.batch_size)):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2]
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            avg = X.mean(axis=0)
            avg.name = 'avg'
            val_min = X.min(axis=0)
            val_min.name = 'min'
            val_max = X.max(axis=0)
            val_max.name = 'max'
            std = X.std(axis=0)
            std.name = 'std'
            last_low = X[-1:] - std
            last_low.name = 'low'
            last_high = X[-1:] + std
            last_high.name = 'high'
            
            #X = pd.concat([X, avg])
            X = X.append(avg)
            X = X.append(val_min)
            X = X.append(val_max)
            X = X.append(std)
            X = X.append(last_low)
            X = X.append(last_high)
            #print('X.shape = ', X.shape)
            #print('x.shape = ', x.shape)
            #X = X.to_numpy().reshape(self.input_width*self.data.shape[1])
            X = X.to_numpy().reshape((-1,))
            if self.selected_features is not None:
                X = X[self.selected_features]
            #Y = Y.to_numpy().reshape(self.label_width*self.out_num_features)
            
            self.batch_id %= self.no
            
            #x = np.append(x, [X], axis = 0)
            #y = np.append(y, [Y], axis = 0)
                
            x[cnt] = X
            y[cnt] = Y.iloc[0]
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**예측 요소 선정**

In [131]:
# 7: chl-a
# 4: TOC
out_features = [7]
# days to use
input_days = 14

In [132]:
# daily averaged data set
rf_train = RFDataGenerator(train_avg, input_width=input_days, shift=5, out_features=out_features)
rf_test = RFDataGenerator(val_avg, input_width=input_days, shift=5, out_features=out_features)

In [133]:
#rf_train = RFDataGenerator(train_df, shift=24*5, out_features=out_features)
#rf_test = RFDataGenerator(val_df, shift=24*5, out_features=out_features)

In [134]:
it = iter(rf_train)
x_train, y_train = next(it)
it = iter(rf_test)
x_test, y_test = next(it)

100%|██████████| 420/420 [00:11<00:00, 35.55it/s]


## Boruta

In [135]:
from sklearn.ensemble import RandomForestRegressor

In [136]:
# n_jobs = CPU core number
forest = RandomForestRegressor(n_estimators=100, n_jobs=16, verbose=1)

In [137]:
forest.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.8s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    3.5s finished


RandomForestRegressor(n_jobs=16, verbose=1)

In [138]:
#print("Score without interactions: {:.3f}".format(
#    rf.score(x_test, y_test)))
print("Accuracy on training set: {:.3f}".format(forest.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(x_test, y_test)))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


Accuracy on training set: 0.978
Accuracy on test set: 0.275


In [139]:
importances = forest.feature_importances_
feature2d = importances.reshape((rf_train.input_width+rf_train.derived_column_features,-1))
feature2d_nan = feature2d.copy()
feature2d_nan[feature2d == 0.] = np.nan
feature2d_log = np.log(feature2d_nan)

In [140]:
# create labels
time_labels = []

for i in range(rf_train.input_width):
    time_labels.append('t-%d'%(-i+rf_train.input_width-1) )
time_labels.append('avg')
time_labels.append('min')
time_labels.append('max')
time_labels.append('std')
time_labels.append('low')
time_labels.append('high')
column_labels = test_avg.columns.to_list()
for i in range(7):
    column_labels[i] = 'T'+column_labels[i]
    column_labels[i+11] = 'S'+column_labels[i+11]

feature_labels = []
for j in range(len(time_labels)):
    for i in range(test_avg.shape[1]):
        feature_labels.append(time_labels[j]+' '+column_labels[i])
feature_labels_np = np.array(feature_labels)

In [141]:
fig, ax = plt.subplots(figsize=(9,7))
ax.imshow(feature2d_log, cmap='rainbow')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
# Loop over data dimensions and create text annotations.
'''
for i in range(len(time_labels)):
    for j in range(len(column_labels)):
        text = ax.text(j, i, '%.3f'%feature2d[i, j],
                       ha="center", va="center", color="w")
'''
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Tree’s Feature Importance from Mean Decrease in Impurity (MDI)**

In [142]:
sorted_idx = importances.argsort()
fig, ax = plt.subplots(figsize=(8,8))
n = 30
#y_ticks = np.arange(0, len(feature_names))
y_ticks = np.arange(0, n)
ax.barh(y_ticks, importances[sorted_idx[-n:]])
ax.set_yticks(y_ticks)
ax.set_yticklabels(feature_labels_np[sorted_idx[-n:]], font_properties=fprop)
fig.tight_layout()
plt.suptitle("Random Forest Feature Importances (MDI)")
plt.show()

#feature_labels_np[sorted_idx[-n:]]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
from boruta import BorutaPy

In [144]:
boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)

**Boruta 학습**

In [146]:
boruta.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.7s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.8s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed:   35.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.8s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed:   35.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:   12.6s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:   30.5s
[Parallel(n_jobs=16)]: Done 481 out of 481 | elapsed: 

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 196 out of 196 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.6s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    2.5s finished
[Parall

BorutaPy(estimator=RandomForestRegressor(n_estimators=194, n_jobs=16,
                                         random_state=RandomState(MT19937) at 0x7FABCB4F7160,
                                         verbose=1),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x7FABCB4F7160)

**Boruta feature map**

In [147]:
green_img = boruta.support_.astype(int).reshape((-1, train_avg.shape[1]))
blue_img = boruta.support_weak_.astype(int).reshape((-1, train_avg.shape[1]))

In [148]:
### print results
#green_area = test_avg.columns[boruta.support_].to_list()
#blue_area = test_avg.columns[boruta.support_weak_].to_list()
#print('features in the green area:', green_area)
#print('features in the blue area:', blue_area)

In [149]:
fig, ax = plt.subplots(figsize=(9,5))
plt.title('Green Features')
plt.imshow(green_img, cmap='Greens')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()
fig, ax = plt.subplots(figsize=(9,5))
plt.title('Blue Features')
plt.imshow(blue_img, cmap='Blues')
# We want to show all ticks...
ax.set_xticks(np.arange(len(column_labels)))
ax.set_yticks(np.arange(len(time_labels)))
# ... and label them with the respective list entries
ax.set_xticklabels(column_labels, fontproperties=fprop)
ax.set_yticklabels(time_labels, fontproperties=fprop)
# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/kotech/workspace/venv-tensor2n-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Random forest

In [150]:
#boruta_features = boruta.support_ | boruta.support_weak_
boruta_features = boruta.support_

In [151]:
np.sum(boruta_features.astype(int))

163

In [152]:
# daily averaged data set
rf_train = RFDataGenerator(train_avg, input_width=input_days, shift=5, out_features=out_features,
                           selected_features=boruta_features)
rf_test = RFDataGenerator(val_avg, input_width=input_days, shift=5, out_features=out_features,
                          selected_features=boruta_features)

In [153]:
it = iter(rf_train)
x_train, y_train = next(it)
it = iter(rf_test)
x_test, y_test = next(it)

100%|██████████| 420/420 [00:11<00:00, 35.48it/s]


In [154]:
forest.fit(x_train, y_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    1.1s finished


RandomForestRegressor(n_estimators=194, n_jobs=16,
                      random_state=RandomState(MT19937) at 0x7FABCB4F7160,
                      verbose=1)

In [155]:
#print("Score without interactions: {:.3f}".format(
#    rf.score(x_test, y_test)))
print("Accuracy on training set: {:.3f}".format(forest.score(x_train, y_train)))
print("Accuracy on test set: {:.3f}".format(forest.score(x_test, y_test)))

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished


Accuracy on training set: 0.984
Accuracy on test set: 0.255


In [156]:
y_pred = forest.predict(x_test)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 194 out of 194 | elapsed:    0.1s finished


In [978]:
std = train_std[out_features[0]]
mean = train_mean[out_features[0]]
y_test_denorm = y_test*std + mean
y_pred_denorm = y_pred*std + mean

In [979]:
norm_mae = np.mean(np.abs(y_pred - y_test))
print('norm_mae =', norm_mae)
mae = np.mean(np.abs(y_pred_denorm - y_test_denorm))
print('mae =', mae)

norm_mae = 0.47531242982703936
mae = 6.905594590176136


In [980]:
plt.figure()
plt.plot(y_test_denorm)
plt.plot(y_pred_denorm)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [981]:
y_pred_train = forest.predict(x_train)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 182 out of 182 | elapsed:    0.1s finished


In [982]:
y_train_denorm = y_train*std + mean
y_pred_train_denorm = y_pred_train*std + mean
plt.figure()
plt.plot(y_train_denorm)
plt.plot(y_pred_train_denorm)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [983]:
plt.figure()
plt.plot(y_train)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Original hour data plot**

In [728]:
chl_a = ori_df.iloc[:, 6]
plt.figure()
plt.plot(chl_a)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 연습섹션

In [635]:
from sklearn.metrics import r2_score
#from scipy import stats
from sklearn.linear_model import LinearRegression

#creating data
x = np.array([0,1,2,3,4,5,6,7,8,9])
y = np.array([0,2,3,5,8,13,21,34,55,89])

#creating OLS regression
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

X = x.reshape((-1, 1))

lr = LinearRegression()
lr.fit(X,y)
y_pred = lr.predict(X)
#def linefitline(b):
#    return intercept + slope * b
#line1 = linefitline(x)

#plot line
plt.figure()
plt.scatter(x,y)
plt.scatter(x,y_pred)
#plt.plot(x,line1, c = 'g')
plt.show()

accuracy = lr.score(X, y)
print('accuracy =', accuracy)
correl = np.corrcoef(y, y_pred)
print('correl =')
print(correl)
print('r2 =', correl[0,1]*correl[0,1])

from sklearn.metrics import r2_score 
R_square = r2_score(y, y_pred) 
print('Coefficient of Determination', R_square) 


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

accuracy = 0.7670001449905756
correl =
[[1.         0.87578544]
 [0.87578544 1.        ]]
r2 = 0.7670001449905752
Coefficient of Determination 0.7670001449905756


least square method에서는 correl^2과 R^2가 동일하다.  
하지만 다른 경우라면...

In [637]:
y = np.array([0,2,3,5,8,13,21,34,55,89])
y_pred = np.array([1,2,7,5,5,12,21,34,55,30])
correl = np.corrcoef(y, y_pred)
print('correl =')
print(correl)
print('correl**2 =', correl[0,1]*correl[0,1])

from sklearn.metrics import r2_score 
R_square = r2_score(y, y_pred) 
print('Coefficient of Determination', R_square) 

correl =
[[1.         0.77879727]
 [0.77879727 1.        ]]
correl**2 = 0.6065251885388013
Coefficient of Determination 0.533758639021797
